# Generators, yield, coroutines
generators save memory, because data is processed one by one, this is useful if a lot of data is to be processed, maybe more than the computer has, or data is not needed all at the same time

## Simple Generator Examples

In [18]:
g = (i for i in range(5))  # looks like tuple comprehension

# but this would be ((i for i in range(5))) 
# or *(i for i in range(5)),
# next(g)

# for i in g: print(i)  # or:
print(next(g), next(g), next(g), next(g), next(g))

0 1 2 3 4


In [8]:
import sys
lst = [i for i in range(1001)]
g = (i for i in range(1001))
print(f'''
    size of list: {sys.getsizeof(lst)}, 
    size of generator: {sys.getsizeof(g)}
    ''')



    size of list: 8856, 
    size of generator: 112


In [12]:
s1 = sum([i for i in range(1001)])
s2 = sum((i for i in range(1001)))  # summe mit generator gebildet
s1, s2

(500500, 500500)

In [4]:
def mygenerator(n):
   for i in range(1, n, 2):
      yield i**3
        
for i in mygenerator(10):
   print(i)

1
27
125
343
729


In [8]:
mygen2 = (i**3 for i in range(1,10,2))
for i in mygen2:
   print(i)

1
27
125
343
729


In [29]:
def mygenerator(n):
   for i in range(1, n, 2):
      yield i * (i + 1)
my_gen = mygenerator(6)
while my_gen:
    try:
        print(next(my_gen))
    except:
        break

2
12
30


In [ ]:
type(my_gen)
my_gen.close()

In [24]:
next(my_gen)

StopIteration: 

In [27]:
myString = iter("Python")
while myString:
    try:
        print(next(myString))
    except StopIteration:
        break

P
y
t
h
o
n


In [25]:
other_cities = ["Strasbourg", "Freiburg", "Stuttgart", 
                "Vienna / Wien", "Hannover", "Berlin", 
                "Zurich"]
city_iterator = iter(other_cities)
while city_iterator:
    try:
        city = next(city_iterator)
        print(city)
    except StopIteration:
        break


Strasbourg
Freiburg
Stuttgart
Vienna / Wien
Hannover
Berlin
Zurich


In [32]:
def fibonacci():
    """Generates an infinite sequence of Fibonacci numbers on demand"""
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b
f = fibonacci()
counter = 0
for x in f:
    print(x, " ", end="")
    counter += 1
    if (counter > 10): 
        break 
print()

0  1  1  2  3  5  8  13  21  34  55  


## yield with throw and send

In [31]:
def is_palindrome(num):
    # Skip single-digit inputs
    if num // 10 == 0:
        return False
    temp = num
    reversed_num = 0

    while temp != 0:
        reversed_num = (reversed_num * 10) + (temp % 10)
        temp = temp // 10

    if num == reversed_num:
        return True
    else:
        return False

def infinite_palindromes():
    num = 0
    while True:
        if is_palindrome(num):
            i = (yield num)
            if i is not None:
                num = i
        num += 1
    
pal_gen = infinite_palindromes()
for i in pal_gen:
    print(i)
    digits = len(str(i))
    if digits == 5:
        pal_gen.throw(ValueError("We don't like large palindromes"))
    pal_gen.send(10 ** (digits))

11
111
1111
10101


ValueError: We don't like large palindromes

## tricky example with yield from

In [1]:
class File:
    def __init__(self, name):
        self.name = name

class Folder(File):
    def __init__(self, name):
        super().__init__(name)
        self.children = []
        
root = Folder("")
etc = Folder("etc")
root.children.append(etc)
etc.children.append(File("passwd"))
etc.children.append(File("groups"))

def walk(file):
    # if type of file is Folder, es werden Folder und files durchlaufen!
    if isinstance(file, Folder):
        yield file.name + "/"
        for f in file.children:
            yield from walk(f)
    else:
        yield file.name

w = walk(root)
next(w)

'/'

In [2]:
next(w)

'etc/'

In [3]:
next(w)

'passwd'